# Interview Summarization

### Setting up Summarization Chain

In [1]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os
# Load environment variables from the .env file
load_dotenv()

api_key = os.getenv("OPEN_AI_KEY")
organization = os.getenv("OPEN_AI_ORG")
# elevenlabs_key = os.getenv("ELEVENLABS_KEY")

llm = OpenAI(
    temperature=0,
    openai_api_key=api_key,
    openai_organization=organization,
)

### Mockup Dialogues

In [3]:
dialogues = [
    { "input": "Tell me about your background and experience.", 
     "output": "I have a degree in computer science and have been working in the tech industry for the past 7 years. My experience includes developing software solutions for various clients and industries, with a focus on web and mobile applications." },
    { "input": "What motivated you to pursue a career in your field?", 
     "output": "I've always been fascinated by technology and its ability to solve real-world problems. This passion led me to pursue a career in computer science, where I could apply my skills to create innovative solutions and contribute to the ever-evolving tech landscape." },
    { "input": "Can you share a challenging project you've worked on and how you overcame obstacles?", "output": "One challenging project involved tight deadlines and complex requirements. I organized a cross-functional team, established clear communication channels, and implemented an agile development approach. This allowed us to adapt quickly to changes and deliver a successful project within the given constraints." },
    { "input": "How do you stay updated with the latest industry trends and technologies?", 
     "output": "I am an avid reader of tech blogs, attend industry conferences, and participate in online forums. Additionally, I allocate time for continuous learning through online courses and certifications to stay abreast of the latest trends, ensuring that my skills are always up-to-date." },
    { "input": "What programming languages and tools are you most proficient in?", 
     "output": "I am proficient in several programming languages, including Python, JavaScript, and Java. I have extensive experience with popular frameworks such as React and Django. Additionally, I am well-versed in using version control systems like Git and have a strong foundation in cloud computing platforms like AWS." },
    { "input": "Describe your approach to problem-solving and decision-making.", 
     "output": "I believe in a systematic approach to problem-solving. I start by thoroughly understanding the problem, breaking it down into manageable components, and then devising a step-by-step plan. I also value input from team members and stakeholders, fostering a collaborative decision-making process that takes diverse perspectives into account." },
    { "input": "How do you handle tight deadlines and high-pressure situations?", 
     "output": "In high-pressure situations, I prioritize tasks, set realistic goals, and communicate transparently with the team. I believe in maintaining a positive attitude and motivating the team to work efficiently. By staying organized and focused, I ensure that we meet deadlines without compromising the quality of our work." },
    { "input": "Tell me about a time when you had to adapt to a significant change in a project.", 
     "output": "During a project, the client's requirements underwent a major shift. I promptly held a team meeting to discuss the changes, reassessed our project plan, and adjusted our strategy accordingly. This flexibility allowed us to accommodate the client's needs while still meeting our project objectives." },
    { "input": "How do you prioritize tasks and manage your time effectively?", 
     "output": "I prioritize tasks based on urgency and importance, utilizing tools like project management software and to-do lists. I set realistic deadlines, allocate time for each task, and regularly reassess priorities. This approach ensures that I can manage my time effectively and consistently meet project milestones." },
    { "input": "What role do communication and collaboration play in your work?", 
     "output": "Communication and collaboration are integral to my work. I maintain open lines of communication with team members, clients, and stakeholders. Regular team meetings, status updates, and clear documentation help ensure that everyone is on the same page, fostering a collaborative and efficient working environment." },
    { "input": "How do you handle feedback, and how do you incorporate it into your work?", 
     "output": "I see feedback as an opportunity for growth. I actively seek constructive feedback from team members and stakeholders, using it to identify areas for improvement. I take a proactive approach to address feedback, refining my work and incorporating valuable insights to enhance the overall quality of the project." },
    { "input": "Tell me about a successful team collaboration experience.", 
     "output": "In a previous project, effective team collaboration played a crucial role in its success. We had diverse skills and perspectives within the team, and by fostering open communication and mutual respect, we were able to capitalize on each team member's strengths. This collaborative effort resulted in a high-quality deliverable that exceeded the client's expectations." },
    { "input": "What do you consider your greatest professional achievement?", 
     "output": "My greatest professional achievement was leading a team in the successful development and launch of a complex e-commerce platform. This project required overcoming technical challenges, coordinating with various stakeholders, and ensuring a seamless user experience. The positive feedback from both the client and end-users was immensely rewarding." },
    { "input": "How do you contribute to creating a positive work culture?", 
     "output": "I believe in fostering a positive work culture through open communication, recognition of achievements, and a supportive team environment. I actively encourage collaboration, celebrate team successes, and provide constructive feedback. By promoting a positive work culture, I aim to create a motivating and enjoyable workplace for everyone on the team." },
    { "input": "What are your professional development goals for the next 12 months?", 
     "output": "In the next 12 months, I aim to deepen my expertise in machine learning and artificial intelligence. I plan to enroll in advanced courses, attend relevant conferences, and undertake hands-on projects to apply my learning. Additionally, I aspire to take on a leadership role within my team, contributing to both technical and strategic aspects of our projects." },
    { "input": "How do you handle disagreements or conflicts within a team?", 
     "output": "I approach conflicts within a team by first seeking to understand the perspectives of all parties involved. I encourage open and honest communication to identify the root causes of the disagreement. By fostering a collaborative environment and finding common ground, I work towards resolving conflicts amicably and maintaining a positive team dynamic." },
    { "input": "What do you enjoy most about your current role?", 
     "output": "I thoroughly enjoy the dynamic nature of my current role. The opportunity to work on diverse projects, tackle new challenges, and collaborate with talented individuals keeps me motivated. The continuous learning and growth within the ever-evolving tech industry make my work both fulfilling and exciting." },
    { "input": "How do you stay motivated and inspired in your work?", 
     "output": "I stay motivated by setting personal and professional goals, staying curious, and celebrating small achievements. I find inspiration in the potential impact of technology on society and the opportunity to contribute to meaningful projects. Additionally, I surround myself with a supportive network of colleagues and mentors, which helps fuel my enthusiasm for continuous learning and improvement." },
    { "input": "Where do you see yourself professionally in the next five years?", 
     "output": "In the next five years, I envision myself in a leadership role, leading innovative projects and contributing to the strategic direction of the organization. I aspire to continue growing as a technical expert while also mentoring and inspiring others within the team. Ultimately, I aim to make a significant impact on both the projects I lead and the professional development of those around me." },
    { "input": "Do you have any questions for us or anything else you'd like to share?", 
     "output": "Yes, I have a couple of questions. Firstly, I'm interested in learning more about the team dynamics and how collaboration is fostered within the company. Additionally, could you provide insights into the company's approach to professional development and growth opportunities for employees?" },
    { "input": "Thank you for sharing your experiences with us. We'll be in touch soon.", 
     "output": "Thank you for the opportunity to discuss my background and experiences. I look forward to the possibility of contributing to the team. If you have any further questions or need additional information, please feel free to reach out. I appreciate your time and consideration." }
]

In [4]:
def generate_interview_content(dialogues):
    formatted_content = ""
    for pair in dialogues:
        formatted_content += f'interview input: {pair["input"]}\n'
        formatted_content += f'candidate output: {pair["output"]}\n\n'

    return formatted_content

# Example usage:
interview_text = generate_interview_content(dialogues)

# Save to a file
with open("interview_conversation.txt", "w") as file:
    interview_text_file = file.write(interview_text)

### Splitter & Chunking

In [5]:
# load the doc
with open("interview_conversation.txt", "r") as file:
    interview_text_file = file.read()

text_splitter = CharacterTextSplitter()

texts = text_splitter.split_text(interview_text_file)
texts

["interview input: Tell me about your background and experience.\ncandidate output: I have a degree in computer science and have been working in the tech industry for the past 7 years. My experience includes developing software solutions for various clients and industries, with a focus on web and mobile applications.\n\ninterview input: What motivated you to pursue a career in your field?\ncandidate output: I've always been fascinated by technology and its ability to solve real-world problems. This passion led me to pursue a career in computer science, where I could apply my skills to create innovative solutions and contribute to the ever-evolving tech landscape.\n\ninterview input: Can you share a challenging project you've worked on and how you overcame obstacles?\ncandidate output: One challenging project involved tight deadlines and complex requirements. I organized a cross-functional team, established clear communication channels, and implemented an agile development approach. Thi

In [6]:
len(texts)

3

In [7]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [8]:
docs

[Document(page_content="interview input: Tell me about your background and experience.\ncandidate output: I have a degree in computer science and have been working in the tech industry for the past 7 years. My experience includes developing software solutions for various clients and industries, with a focus on web and mobile applications.\n\ninterview input: What motivated you to pursue a career in your field?\ncandidate output: I've always been fascinated by technology and its ability to solve real-world problems. This passion led me to pursue a career in computer science, where I could apply my skills to create innovative solutions and contribute to the ever-evolving tech landscape.\n\ninterview input: Can you share a challenging project you've worked on and how you overcame obstacles?\ncandidate output: One challenging project involved tight deadlines and complex requirements. I organized a cross-functional team, established clear communication channels, and implemented an agile dev

#  3 types of CombineDocuments Chains

[Taken from the LangChain Docs](https://langchain.readthedocs.io/en/latest/modules/indexes/combine_docs.html)

## 1. Summarize Simple with map_reduce

### Map Reduce
This method involves `an initial prompt on each chunk of data`
( for summarization tasks, this could be a summary of that chunk; for question-answering tasks, it could be an answer based solely on that chunk). `Then a different prompt is run to combine all the initial outputs.` This is implemented in the LangChain as the MapReduceDocumentsChain.

`Pros:` Can scale to larger documents (and more documents) than StuffDocumentsChain. The calls to the LLM on individual documents are independent and can therefore be parallelized.

`Cons:` Requires many more calls to the LLM than StuffDocumentsChain. Loses some information during the final combining call.

In [10]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

chain = load_summarize_chain(
    llm, 
    chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

 The candidate has a degree in computer science and 7 years of experience in the tech industry. They
are proficient in multiple programming languages and tools, and have a systematic approach to
problem-solving and decision-making. They prioritize tasks, manage their time effectively, and
maintain open lines of communication and collaboration with team members, clients, and stakeholders.
The candidate expressed their ambition to take on a leadership role in the next five years, while
continuing to grow as a technical expert and mentor others.


In [11]:
# for summarizing each part
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [12]:
# for combining the parts
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [13]:
chain = load_summarize_chain(
    llm, 
    chain_type="map_reduce",
    verbose=True
)

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(
    output_summary, 
    width=100,
    break_long_words=False,
    replace_whitespace=False)

print(wrapped_text)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"interview input: Tell me about your background and experience.
candidate output: I have a degree in computer science and have been working in the tech industry for the past 7 years. My experience includes developing software solutions for various clients and industries, with a focus on web and mobile applications.

interview input: What motivated you to pursue a career in your field?
candidate output: I've always been fascinated by technology and its ability to solve real-world problems. This passion led me to pursue a career in computer science, where I could apply my skills to create innovative solutions and contribute to the ever-evolving tech landscape.

interview input: Can you share a challenging project you've worked on and how you overcame obstacles?
candidate output: One challenging project involved tight deadlines and compl

## 2. Summarizing with the 'stuff' Chain

### Stuffing
Stuffing is the simplest method, whereby you simply stuff all the related data into the prompt as context to pass to the language model. This is implemented in LangChain as the StuffDocumentsChain.

`Pros:` Only makes a single call to the LLM. When generating text, the LLM has access to all the data at once.

`Cons:` Most LLMs have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.

The main downside of this method is that `it only works one smaller pieces of data.`  
- Once you are working with many pieces of data, this approach is no longer feasible. 
- The next two approaches are designed to help deal with that.

In [14]:
chain = load_summarize_chain(
    llm, 
    chain_type="stuff")

In [15]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])


In [16]:
chain = load_summarize_chain(
    llm, 
    chain_type="stuff", 
    prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(
    output_summary, 
    width=100,
    break_long_words=False,
    replace_whitespace=False)
print(wrapped_text)


• Degree in computer science and 7 years of experience in tech industry
• Passion for technology
and problem-solving drives career in computer science
• Proficient in Python, JavaScript, Java,
React, Django, Git, and AWS
• Systematic approach to problem-solving and decision-making
•
Prioritizes tasks and manages time effectively
• Values communication and collaboration in work
•
Adapts to changes and incorporates feedback
• Experienced in leading successful projects and teams
•
Motivated by setting goals and celebrating achievements
• Seeks to deepen expertise in machine
learning and AI
• Contributes to positive work culture and professional development
• Aspires to
leadership role in next 5 years


### With 'map_reduce' with our custom prompt

In [17]:
chain = load_summarize_chain(
    llm, 
    chain_type="map_reduce",
    map_prompt=BULLET_POINT_PROMPT, 
    combine_prompt=BULLET_POINT_PROMPT)

# chain.llm_chain.prompt= BULLET_POINT_PROMPT
# chain.combine_document_chain.llm_chain.prompt= BULLET_POINT_PROMPT

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(
    output_summary, 
    width=100,
    break_long_words=False,
    replace_whitespace=False)

print(wrapped_text)


• Degree in computer science and 7 years of experience in tech industry
• Proficient in Python,
JavaScript, Java, React, Django, Git, and AWS
• Systematic approach to problem-solving and decision-
making
• Prioritizes tasks and manages time effectively
• Values communication and collaboration for
efficient working environment
• Led successful development and launch of a complex e-commerce
platform
• Promotes positive work culture through open communication, recognition of achievements,
and a supportive team environment
• Aims to deepen expertise in machine learning and artificial
intelligence in the next 12 months
• Handles disagreements or conflicts within a team by seeking to
understand perspectives and fostering a collaborative environment
• Candidate envisions themselves
in a leadership role in the next five years, leading innovative projects and contributing to the
organization's strategic direction


In [24]:
# with a custom prompt
prompt_template = """Write a concise summary of the following:

{text}

CONSCISE SUMMARY IN BULLET POINTS:"""

PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

## with intermediate steps
chain = load_summarize_chain(
    llm, 
    chain_type="map_reduce", 
    return_intermediate_steps=True, 
    map_prompt=PROMPT, 
    combine_prompt=PROMPT)

output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(
    output_summary['output_text'], 
    width=100,
    break_long_words=False,
    replace_whitespace=False
)
print(wrapped_text)


• The candidate has a degree in computer science and 7 years of experience in the tech industry,
developing software solutions for various clients and industries.
• Proficient in programming
languages such as Python, JavaScript, and Java, and have experience with popular frameworks such as
React and Django.
• Systematic approach to problem-solving and decision-making, and prioritize tasks
and manage their time effectively.
• Values communication and collaboration, and maintains open
lines of communication with team members, clients, and stakeholders.
• Led a successful team
collaboration experience, resulting in a high-quality deliverable that exceeded the client's
expectations.
• Seeks constructive feedback from team members and stakeholders.
• Envisions
themselves in a leadership role in the next five years, leading innovative projects and contributing
to the strategic direction of the organization.


In [19]:
wrapped_text = textwrap.fill(
    output_summary['intermediate_steps'][2], 
    width=100,
    break_long_words=False,
    replace_whitespace=False)
print(wrapped_text)



• Candidate envisions themselves in a leadership role in the next five years, leading innovative
projects and contributing to the organization's strategic direction.
• Asks questions about team
dynamics, collaboration, and professional development opportunities.
• Expresses appreciation for
the opportunity to discuss their background and experiences.


## 3. With the 'refine' CombineDocument Chain

### Refine
This method involves `an initial prompt on the first chunk of data, generating some output. For the remaining documents, that output is passed in, along with the next document`, asking the LLM to refine the output based on the new document.

`Pros:` Can pull in more relevant context, and may be less lossy than MapReduceDocumentsChain.

`Cons:` Requires many more calls to the LLM than StuffDocumentsChain. The calls are also NOT independent, meaning they cannot be paralleled like MapReduceDocumentsChain. There is also some potential dependencies on the ordering of the documents.

In [20]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  The candidate has a degree in computer science and 7 years of experience in the tech industry,
developing software solutions for various clients and industries. They are passionate about
technology and its ability to solve real-world problems, and stay up-to-date with the latest
industry trends and technologies through reading tech blogs, attending conferences, and taking
online courses. They are proficient in multiple programming languages and tools, and have a
systematic approach to problem-solving and decision-making. They prioritize tasks, manage their time
effectively, and maintain open lines of communication and collaboration with team members, clients,
and stakeholders. They actively seek constructive feedback from team members and stakeholders, using
it to identify areas for improvement and refine their work. They have a proactive approach to
addressing feedback, and have successfully led teams in the development and launch of complex
e-commerce platforms. They prioritize cre

In [21]:
prompt_template = """Write a concise summary of the following extracting the key information:

{text}

CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(
    llm, 
    chain_type="refine", 
    return_intermediate_steps=True, 
    question_prompt=PROMPT, 
    refine_prompt=refine_prompt)


In [22]:
output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(
    output_summary['output_text'], 
    width=100,
    break_long_words=False,
    replace_whitespace=False)

print(wrapped_text)



The candidate has a degree in computer science and 7 years of experience in the tech industry,
developing software solutions for various clients and industries. They are passionate about
technology and its ability to solve real-world problems, and stay up-to-date with the latest
industry trends and technologies through reading tech blogs, attending conferences, and taking
online courses. They are proficient in programming languages such as Python, JavaScript, and Java,
and have experience with frameworks such as React and Django. They use a systematic approach to
problem-solving, prioritize tasks, and maintain open lines of communication and collaboration with
team members, clients, and stakeholders. They are able to adapt to changes and manage their time
effectively to meet deadlines. They actively seek constructive feedback from team members and
stakeholders, using it to identify areas for improvement and take a proactive approach to address
feedback, refining their work and incorp

In [23]:
wrapped_text = textwrap.fill(
    output_summary['intermediate_steps'][0], 
    width=100,
    break_long_words=False,
    replace_whitespace=False
)
print(wrapped_text)

 The candidate has a degree in computer science and 7 years of experience in the tech industry,
developing software solutions for various clients and industries. They are passionate about
technology and its ability to solve real-world problems, and stay up-to-date with the latest
industry trends and technologies through reading tech blogs, attending conferences, and taking
online courses. They are proficient in programming languages such as Python, JavaScript, and Java,
and have experience with frameworks such as React and Django. They use a systematic approach to
problem-solving, prioritize tasks, and maintain open lines of communication and collaboration with
team members, clients, and stakeholders. They are able to adapt to changes and manage their time
effectively to meet deadlines.
